### Import packages

In [1]:
import matplotlib.pyplot as plt
!matplotlib inline

/bin/bash: matplotlib: command not found


In [2]:
from pathlib import Path
import time

import pandas as pd
import numpy as np
import gym
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

import d3rlpy

/home/orogov/improved-diffusion/anaconda3/envs/rl/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Read and preprocess data

In [3]:
data_dir = Path('data')

train = pd.read_csv(data_dir / 'train_data.csv')
test = pd.read_csv(data_dir / 'test_data.csv')

In [4]:
def fill_unknown_previous_rate(data, mean=None):
    """Estimate unknown previous rate with mean value
    """
    data['Previous_Rate_Known'] = (data['Previous_Rate'] != -10)
    
    prev_rate = data['Previous_Rate'].copy()
    if mean:
        prev_rate_mean = mean
    else:
        prev_rate_mean = prev_rate[prev_rate != -10].mean()
    
    prev_rate[prev_rate == -10] = prev_rate_mean
    data['Previous_Rate'] = prev_rate
    
    return prev_rate_mean

prev_rate_mean = fill_unknown_previous_rate(train)
fill_unknown_previous_rate(test, prev_rate_mean)

7.625268573157815

In [5]:
# samples
train.head(3)

,Tier,FICO,Term,Amount,Previous_Rate,Competition_rate,Rate,Cost_Funds,Partner Bin,Car_Type_N,Car_Type_R,Car_Type_U,Accept,Previous_Rate_Known
0,2,725,72,30500.0,5.000000,6.09,4.99,1.120,2,0,1,0,0,True
1,1,739,60,25995.0,7.625269,4.79,4.79,1.959,2,0,0,1,0,False
2,1,781,60,39000.0,7.625269,4.25,4.25,1.120,2,1,0,0,0,False


In [6]:
# statistics for training data
train.describe()

,Tier,FICO,Term,Amount,Previous_Rate,Competition_rate,Rate,Cost_Funds,Partner Bin,Car_Type_N,Car_Type_R,Car_Type_U,Accept
count,145659.000000,145659.000000,145659.000000,145659.000000,145659.000000,145659.000000,145659.000000,145659.000000,145659.000000,145659.000000,145659.000000,145659.000000,145659.000000
mean,1.928429,726.695556,56.822441,25996.819760,7.625269,4.807957,5.623998,1.329209,2.029473,0.571815,0.226865,0.201320,0.220041
std,1.050519,44.727756,11.201037,11125.968255,1.323102,0.586018,1.545418,0.278368,0.911097,0.494817,0.418806,0.400988,0.414276
min,1.000000,594.000000,36.000000,4526.620000,3.000000,2.990000,2.450000,1.020000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,692.000000,48.000000,17753.450000,7.625269,4.390000,4.490000,1.110000,1.000000,0.000000,0.000000,0.000000,0.000000
50%,2.000000,726.000000,60.000000,25000.000000,7.625269,4.790000,5.090000,1.262500,2.000000,1.000000,0.000000,0.000000,0.000000
75%,3.000000,762.000000,60.000000,33000.000000,7.625269,5.190000,6.390000,1.419400,3.000000,1.000000,0.000000,0.000000,0.000000
max,4.000000,852.000000,72.000000,100000.000000,24.000000,6.450000,15.530000,2.127000,3.000000,1.000000,1.000000,1.000000,1.000000


### Train response probabilities parametrized by logistic regression model

In [7]:
X = train[['Tier', 'FICO', 'Term', 'Amount', 'Previous_Rate', 'Competition_rate',
           'Cost_Funds', 'Partner Bin', 'Car_Type_N', 'Car_Type_R', 'Car_Type_U', 'Rate']].values
y = train['Accept'].values

# train
accept_model = LogisticRegression(max_iter=300, penalty=None, fit_intercept=True, 
                                  multi_class='ovr', random_state=42, n_jobs=4).fit(X, y)

### Split on train/eval, prepare (observations, actions) data

In [8]:
def split_on_observations_nd_actions(data: pd.DataFrame):
    observations = data[['Tier', 'FICO', 'Term', 'Amount', 'Previous_Rate', 'Competition_rate',
                         'Cost_Funds', 'Partner Bin', 'Car_Type_N', 'Car_Type_R', 'Car_Type_U']]
    actions = data['Rate']

    observations = observations.values
    actions = actions.values.reshape(-1, 1)

    return observations, actions

In [9]:
n_samples = 500  # choose number of samples for overall training/evaluation. Training time and accuracy depends on this
test_size = 0.1

train_data, eval_data = train_test_split(train[:n_samples], test_size=test_size, shuffle=True, random_state=42)

In [10]:
train_observations, train_actions = split_on_observations_nd_actions(train_data)
eval_observations, eval_actions = split_on_observations_nd_actions(eval_data)

### Define reward function

In [11]:
def p_default(state):
    fico = state[:, 1]
    
    default_prob = fico.copy()
    default_prob[default_prob < 500] = 0.41
    default_prob[default_prob >= 750] = 0.01
    default_prob[default_prob >= 700] = 0.044
    default_prob[default_prob >= 650] = 0.089
    default_prob[default_prob >= 600] = 0.158
    default_prob[default_prob >= 550] = 0.225
    default_prob[default_prob >= 500] = 0.284

    return default_prob


def calc_accept_prob(action, state, model):
    if isinstance(action, (int, float)):
        action = np.array(action)

    action = action.reshape(-1, 1)
    
    if len(state.shape) == 1:
        state = state.reshape(1, -1)
    
    x = np.concatenate((state, action), axis=1)
    probs = model.predict_proba(x)[:, 1]
    
    return probs


def reward(action, state, model, risk_free=0.2, loss_ratio=0.5):
    if len(state.shape) == 1:
        state = state.reshape(1, -1)

    p_accept= calc_accept_prob(action, state, model)

    Sum_loan = state[:, 3]
    Term = state[:, 2] / 12

    p_return = p_default(state)

    loss_given_default = loss_ratio * Sum_loan
    action = action / 100
    reward = (p_accept*(Sum_loan*p_return*((1+action)**Term-(1+risk_free)**Term)-(1-p_return)*loss_given_default))/Sum_loan

    return reward

In [12]:
# calculate train and evaluation rewards

train_rewards = reward(train_actions, train_observations, accept_model, risk_free=0.04, loss_ratio=0.5)
eval_rewards = reward(eval_actions, eval_observations, accept_model, risk_free=0.04, loss_ratio=0.5)

In [13]:
class Environment(gym.Env):
    def __init__(self, dataset, accept_model):
        super().__init__()

        self.dataset = dataset
        self.accept_model = accept_model
        
        self.curr_state = 0
        self.n_observation = dataset.shape[0]
        
        observation_tensor = dataset[['Tier', 'FICO', 'Term', 'Amount', 'Previous_Rate', 'Competition_rate',
                                      'Cost_Funds', 'Partner Bin', 'Car_Type_N', 'Car_Type_R','Car_Type_U']]
        
        self.observation_tensor = observation_tensor.values
        self.action_space = gym.spaces.Box(low=np.array([0]), high=np.array([100]))
        self.observation_space = gym.spaces.Box(low=-10*np.ones(11)[None, :], 
                                                high=10e+10*np.ones(11)[None, :], 
                                                shape=(1, 11))
    
    def step(self, action):
        """
          1. Update state
          2. Return reward

          Return format:
            next_s, r, done, info = self.step(a)
        """
        state = self.observation_tensor[self.curr_state]
       
        if self.curr_state >= self.n_observation - 2:
            done = True
        else:
            done = False
        
        self.curr_state +=1
        next_s = self.observation_tensor[self.curr_state]
        r = reward(action, state, self.accept_model, risk_free=0.04, loss_ratio=0.5)

        info = f'reward = {r}, at state is done = {done}'

        return next_s, r, done, info

    def reset(self):
        self.curr_state = 0
        state = self.observation_tensor[self.curr_state]
        
        return state

### Training

In [14]:
# Initialize environment
environment = Environment(train_data, accept_model)

# Initialize train dataset
train_dataset = d3rlpy.dataset.MDPDataset(
    observations=train_observations,
    actions=train_actions,
    rewards=train_rewards,
    terminals=np.random.randint(2, size=train_actions.shape[0]),
)

# Initialize evaluation dataset
eval_dataset =d3rlpy.dataset.MDPDataset(
    observations=eval_observations,
    actions=eval_actions,
    rewards=eval_rewards,
    terminals=np.zeros_like(eval_actions)
)

# Initialize CQL algorithm
cql = d3rlpy.algos.CQL(use_gpu=True, n_steps=5, batch_size=256, gamma=0.999,
                       n_critics=2, alpha_threshold=10, conservative_weight=5)

print(f'Action size = ', train_dataset.get_action_size())
time.sleep(0.1)

# Train
n_epochs = 5  # set number of epochs
cql.fit(train_dataset,
        eval_episodes=train_dataset,
        n_epochs=n_epochs,
        scorers={'environment': d3rlpy.metrics.evaluate_on_environment(environment),
                 'td_error': d3rlpy.metrics.td_error_scorer,},)

Action size =  1
2023-10-22 21:07:26 [debug    ] RoundIterator is selected.
2023-10-22 21:07:26 [info     ] Directory is created at d3rlpy_logs/CQL_20231022210726
2023-10-22 21:07:26 [debug    ] Building models...


/home/orogov/improved-diffusion/anaconda3/envs/rl/lib/python3.8/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


2023-10-22 21:07:31 [debug    ] Models have been built.
2023-10-22 21:07:31 [info     ] Parameters are saved to d3rlpy_logs/CQL_20231022210726/params.json params={'action_scaler': None, 'actor_encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': None}}, 'actor_learning_rate': 0.0001, 'actor_optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}, 'alpha_learning_rate': 0.0001, 'alpha_optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}, 'alpha_threshold': 10, 'batch_size': 256, 'conservative_weight': 5, 'critic_encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': None}}, 'critic_learning_rate': 0.0003, 'critic_optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}, 'gamma': 0.999, 'generated_maxlen': 

Epoch 1/5: 100%|██████████| 1/1 [00:01<00:00,  1.10s/it, temp_loss=-1.24, temp=1, alpha_loss=-10.3, alpha=1, critic_loss=97.1, actor_loss=2.04]


2023-10-22 21:08:16 [info     ] CQL_20231022210726: epoch=1 step=1 epoch=1 metrics={'time_sample_batch': 0.00047278404235839844, 'time_algorithm_update': 1.0983357429504395, 'temp_loss': -1.2423245906829834, 'temp': 1.000100016593933, 'alpha_loss': -10.27949333190918, 'alpha': 1.000100016593933, 'critic_loss': 97.14276123046875, 'actor_loss': 2.0415968894958496, 'time_step': 1.100050926208496, 'environment': -119.73413711066466, 'td_error': 6.34205901748491} step=1
2023-10-22 21:08:16 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20231022210726/model_1.pt


...

Epoch 4/5: 100%|██████████| 1/1 [00:00<00:00, 10.98it/s, temp_loss=0.28, temp=1, alpha_loss=-9.82, alpha=1, critic_loss=29.4, actor_loss=1.33]


2023-10-22 21:10:29 [info     ] CQL_20231022210726: epoch=4 step=4 epoch=4 metrics={'time_sample_batch': 0.0010001659393310547, 'time_algorithm_update': 0.0888662338256836, 'temp_loss': 0.27996501326560974, 'temp': 1.000361680984497, 'alpha_loss': -9.823066711425781, 'alpha': 1.0003997087478638, 'critic_loss': 29.352161407470703, 'actor_loss': 1.3291139602661133, 'time_step': 0.09093284606933594, 'environment': -119.73413711066466, 'td_error': 9.81177928168372} step=4
2023-10-22 21:10:29 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20231022210726/model_4.pt


Epoch 5/5: 100%|██████████| 1/1 [00:00<00:00,  9.07it/s, temp_loss=-.824, temp=1, alpha_loss=-9.66, alpha=1, critic_loss=80, actor_loss=2.83]


2023-10-22 21:11:14 [info     ] CQL_20231022210726: epoch=5 step=5 epoch=5 metrics={'time_sample_batch': 0.0005130767822265625, 'time_algorithm_update': 0.10753583908081055, 'temp_loss': -0.8236473798751831, 'temp': 1.000435471534729, 'alpha_loss': -9.661993026733398, 'alpha': 1.0004993677139282, 'critic_loss': 79.95272064208984, 'actor_loss': 2.8345885276794434, 'time_step': 0.10919308662414551, 'environment': -95.19406617467935, 'td_error': 5.935430371984511} step=5
2023-10-22 21:11:14 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20231022210726/model_5.pt


[(1,
  {'time_sample_batch': 0.00047278404235839844,
   'time_algorithm_update': 1.0983357429504395,
   'temp_loss': -1.2423245906829834,
   'temp': 1.000100016593933,
   'alpha_loss': -10.27949333190918,
   'alpha': 1.000100016593933,
   'critic_loss': 97.14276123046875,
   'actor_loss': 2.0415968894958496,
   'time_step': 1.100050926208496,
   'environment': -119.73413711066466,
   'td_error': 6.34205901748491}),
 (2,
  {'time_sample_batch': 0.0006546974182128906,
   'time_algorithm_update': 0.07401728630065918,
   'temp_loss': -1.4116098880767822,
   'temp': 1.0002001523971558,
   'alpha_loss': -10.128416061401367,
   'alpha': 1.0001999139785767,
   'critic_loss': 33.96532440185547,
   'actor_loss': 2.889057159423828,
   'time_step': 0.0757455825805664,
   'environment': -119.73413711066466,
   'td_error': 2.3122149630685778}),
 (3,
  {'time_sample_batch': 0.0009629726409912109,
   'time_algorithm_update': 0.09857034683227539,
   'temp_loss': -0.6275063753128052,
   'temp': 1.000293

### Inference

In [15]:
test_observations, test_actions = split_on_observations_nd_actions(test)

In [16]:
test_observations.shape, test_actions.shape

((62426, 11), (62426, 1))

In [21]:
# use policy  and collect reward??
# How to check???

test_rewards = reward(test_actions[:5000], test_observations[:5000], accept_model, risk_free=0.04, loss_ratio=0.5)
env = Environment(test, accept_model)
state = env.reset()
rewards_cql = []

for step in range(10):
    actions = cql.predict(state.reshape(1, -1))
    next_state, r, done, info = env.step(actions)  # test actions or actions???
    rewards_cql.append(r)
    state = next_state

# hist_rewards_test = reward_vec(test_actions, test_states, risk_free = 0.04, loss_ratio=0.5)
# env  = Env_rl_bank(test_data)
# state = env.reset()
# rewards_cql = []
# for step_i in range(len(test_data)):
#     actions = cql.predict(state)
#     next_state, rew, done, info = env.step(test_actions)
#     rewards_cql.append(rew)
#     state = next_state




/home/orogov/improved-diffusion/anaconda3/envs/rl/lib/python3.8/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [23]:
cum_sum_rew_cql = np.cumsum(rewards_cql)
# cum_sum_rew_hist = np.cumsum(hist_rewards_test)

In [30]:
cum_sum_rew_hist, cum_sum_rew_cql

(array([-1.47296613e-03, -1.14274973e-01, -1.19916873e-01, ...,
        -2.77118592e+03, -2.77119159e+03, -2.77133017e+03]),
 array([], dtype=float64))